In [1]:
from shapely.geometry import LineString, Point, MultiPoint, Polygon

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import geopandas as gpd
import numpy as np
from datetime import datetime
from shapely.geometry import LineString, Point, MultiPoint, Polygon
from IPython.display import display, clear_output

df = pd.read_csv('data/train.csv')

starts = gpd.GeoDataFrame(df[['id']], geometry=gpd.points_from_xy(df.pickup_longitude, df.pickup_latitude), crs=4326)
stops = gpd.GeoDataFrame(df[['id']], geometry=gpd.points_from_xy(df.dropoff_longitude, df.dropoff_latitude), crs=4326)

path_to_data = gpd.datasets.get_path("nybb")
gdf = gpd.read_file(path_to_data)
manh = gdf.loc[gdf.BoroName=='Manhattan'].to_crs(4326)

# roads = pd.read_pickle('roadsfornx.pkl')

sind = gpd.tools.sjoin(starts, manh, op='within').index.tolist()
eind = gpd.tools.sjoin(stops, manh, op='within').index.tolist()
df = df.loc[list(set(sind).intersection(set(eind)))]
starts = gpd.GeoDataFrame(df[['id']], geometry=gpd.points_from_xy(df.pickup_longitude, df.pickup_latitude), crs=4326)
stops = gpd.GeoDataFrame(df[['id']], geometry=gpd.points_from_xy(df.dropoff_longitude, df.dropoff_latitude), crs=4326)

# reps = pd.read_pickle('reps.pkl')

In [11]:
from tqdm.auto import tqdm

tqdm.pandas()

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from routing import *
from misc import viz2

In [4]:
rnds, reps = create_node_info()

Road directions - starts/stops
Clustering
creating nodes


  0%|          | 0/13665 [00:00<?, ?it/s]

In [5]:
G = create_graph(rnds, reps)

In [6]:
stops

,id,geometry
0,id2875421,POINT (-73.96463 40.76560)
1,id2377394,POINT (-73.99948 40.73115)
2,id3858529,POINT (-74.00533 40.71009)
3,id3504673,POINT (-74.01227 40.70672)
4,id2181028,POINT (-73.97292 40.78252)
...,...,...
1458639,id2376096,POINT (-73.99491 40.74017)
1458640,id1049543,POINT (-73.97018 40.79655)
1458641,id2304944,POINT (-74.00443 40.70737)
1458642,id2714485,POINT (-73.97463 40.75711)


In [16]:
def apply_route(row, G, reps):
    st = Point(row.pickup_longitude, row.pickup_latitude)
    en = Point(row.dropoff_longitude, row.dropoff_latitude)

    return Groute(G, st, en, reps)

In [17]:
df.progress_apply(lambda a: apply_route(a, G, reps), axis=1)

  0%|          | 0/1232289 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [44]:
rts = []
for i in tqdm(range(1232289)):

    rts.append(Groute(G, starts.iloc[i].geometry, stops.iloc[i].geometry, reps))

    # viz2(rt, starts.iloc[i:i+1], stops.iloc[i:i+1])
    # input()
    # clear_output()


  0%|          | 0/1232289 [00:00<?, ?it/s]

In [14]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
